# Context

Our bidding strategies run multiple optimization models during a single bidding time period, for instance for different expected prices. At the moment, this is done from scratch each time, but often only a few things from an optimization model change.

This notebook contains some explorations for persistent solvers
1. Example of using persistent solver when updating constraints on a model, and time it.
2. Example for model with updating time-indexed parameters in the objective function, as is done in the `OptimizerBiddingStrategy`.


## 1. Simple model

In [1]:
import pyomo.environ as pyo

In [2]:
def create_model():
    """Create simple model."""
    m = pyo.ConcreteModel()
    m.x = pyo.Var()
    m.y = pyo.Var()
    m.obj = pyo.Objective(expr=m.x**2 + m.y**2)
    m.c = pyo.Constraint(expr=m.y >= -2 * m.x + 5)
    return m

In [3]:
def solve_model(model, solver="gurobi", show_result=True):
    """Solve simple model."""
    opt = pyo.SolverFactory(solver)
    if solver == "gurobi_persistent":
        save_results = False  # in this case, unclear what's better
        opt.set_instance(model)
        _ = opt.solve(save_results=save_results)
        model.c2 = pyo.Constraint(expr=m.y >= m.x)
        opt.add_constraint(m.c2)
        _ = opt.solve(save_results=save_results)
        opt.remove_constraint(m.c2)
        del m.c2
    else:
        _ = opt.solve(model)
        model.c2 = pyo.Constraint(expr=m.y >= m.x)
        _ = opt.solve(model)
        del m.c2

    if show_result:
        print(pyo.value(m.obj))

First, verify we get the same results

In [4]:
m = create_model()
res = solve_model(m, "gurobi_persistent")
res

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2684518
Academic license 2684518 - for non-commercial use only - registered to f.___@tudelft.nl
5.555555556979119


In [5]:
m = create_model()
res = solve_model(m, "gurobi")

5.555555556979119


Now, compare the timing

In [6]:
m = create_model()
%timeit res = solve_model(m, "gurobi_persistent", show_result=False)

11.5 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
m = create_model()
%timeit res = solve_model(m, "gurobi", show_result=False)

22.2 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


To generalize, we need to understand how we can modify different parts of a model


## 2. Docs
- [`GurobiPersistent`](https://pyomo.readthedocs.io/en/stable/reference/topical/solvers/gurobi_persistent.html#gurobipersistent)
- [`AbstractScalarVar`](https://pyomo.readthedocs.io/en/stable/api/pyomo.core.base.var.AbstractScalarVar.html#pyomo.core.base.var.AbstractScalarVar)
- Persistent Solver explanation: [link](https://pyomo.readthedocs.io/en/stable/explanation/solvers/persistent.html#persistent-solvers)

### From the Docs
- "Note that users are responsible for notifying the persistent solver interfaces when changes are made to the corresponding pyomo model."
- `GurobiPersistent` accepts only a ConcreteModel, not an abstract model (!)
- Key methods on the class
    - `add_block`, `remove_block`
    - `add_column`, -- **no method to remove columns**
    - `add_constraint`, `remove_constraint`
    - `add_var`, `remove_var`, `update_var`
    - `set_objective`
- It's necessary to first remove the component from the optimizer, then update the component in pyomo, and then add it back


For instance, constraints should be changed as follows

```python
m = pyo.ConcreteModel()
m.x = pyo.Var()
m.y = pyo.Var()
m.c = pyo.Constraint(expr=m.y >= -2*m.x + 5)
opt = pyo.SolverFactory('gurobi_persistent')
opt.set_instance(m)

opt.remove_constraint(m.c)
del m.c
m.c = pyo.Constraint(expr=m.y <= m.x)
opt.add_constraint(m.c)

```


An exception to this rule are variables, which we can update directly with the solver
```python
m = pyo.ConcreteModel()
m.x = pyo.Var()
m.y = pyo.Var()
m.c = pyo.Constraint(expr=m.y >= -2*m.x + 5)
opt = pyo.SolverFactory('gurobi_persistent')
opt.set_instance(m)

m.x.setlb(1.0) # first, update in pyomo - I understand this 
opt.update_var(m.x)
```

#### Indexed variables and constraints

To add and remove indexed variables and constraints, we need to iterate over their values:
```python
for v in indexed_var.values():
    opt.add_var(v)
    opt.remove_var(v)
for v in indexed_con.values():
    opt.add_constraint(v)
    opt.remove_constraint(v)
```




## 2. Time-indexed parameters to objective function, similar to OptimizerBiddingStrategy

In [8]:
parameters = {"first": {1: 0.5, 2: 3.4}, "second": {1: 5, 2: 3.4}}

In [9]:
def create_model(params_obj_fct):
    """Create a model with time index."""
    model = pyo.ConcreteModel()
    model.time = pyo.Set(initialize=[1, 2])
    model.x = pyo.Var(model.time)

    @model.Param(model.time, domain=pyo.Any, mutable=True)
    def obj_params(model, t):
        return params_obj_fct[t]

    @model.Objective()
    def obj(model):
        return sum(model.obj_params[t] * model.x[t] ** 2 for t in model.time)

    @model.Constraint()
    def c(model):
        return model.x[2] >= -2 * model.x[1] + 5

    return model


def update_model(model, params_obj_fct):
    """Update parameters of obj fct in existing model."""
    del model.obj_params
    del model.obj

    @model.Param(model.time, domain=pyo.Any, mutable=True)
    def obj_params(model, t):
        return params_obj_fct[t]

    @model.Objective()
    def obj(model):
        return sum(model.obj_params[t] * model.x[t] ** 2 for t in model.time)

    return model

In [10]:
def solve_and_update(params, show_results=True):
    """Create a model, solve, change parameters, solve again."""
    model = create_model(params["first"])
    opt = pyo.SolverFactory("gurobi")
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))
    model = create_model(params["second"])
    opt.solve(model)
    if show_results:
        print(pyo.value(model.obj))

In [11]:
solve_and_update(parameters)

3.0141843971849
22.84946236650336


In [12]:
%timeit solve_and_update(parameters, False)

20.5 ms ± 797 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Using a persistent solver

In [13]:
def solve_persist_and_update(params, show_results=True):
    """Create model, solve, change parameters only in solver, solve again."""
    model = create_model(parameters["first"])
    opt = pyo.SolverFactory("gurobi_persistent")
    opt.set_instance(model)
    _ = opt.solve(save_results=False)
    if show_results:
        print(pyo.value(model.obj))

    model = update_model(model, parameters["second"])
    opt.set_objective(model.obj)
    _ = opt.solve(save_results=False)
    if show_results:
        print(pyo.value(model.obj))

In [14]:
solve_persist_and_update(parameters)

3.0141843971849
22.84946236650336


In [15]:
%timeit solve_persist_and_update(parameters, False)

13 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
